In [ ]:
#批量把文本描述对应生成到单个txt文件 一行描述
# text descriptions
import os
import json
total_des=r"F:\fsy\project\kae_process\kae_text\sad.json"
data = r"F:\fsy\project\kae_process\sad\all.txt"
text_dir = r"F:\fsy\project\kae_process\kae_text\sad"
os.makedirs(text_dir, exist_ok=True)


with open(total_des,"r",encoding="utf-8") as f:
    all_texts = json.load(f)
with open(data,"r") as f:
    lines = [ln.strip() for ln in f.readlines()]

assert len(lines) == len(all_texts), f"数量对不上: {len(lines)} vs {len(all_texts)}"
    
for idx, name in enumerate(lines):
    descriptions = [all_texts[idx]]  # 未来如果变成 list[list[str]]，直接 = all_texts[idx] 即可
    out_file = os.path.join(text_dir, name + ".txt")
    with open(out_file, "w", encoding="utf-8") as out_f:
        for desc in descriptions:
            out_f.write(desc + "\n")

In [ ]:
#多行描述
import os
import json

data = r"F:\fsy\project\kae_process\sad\all.txt"
text_dir = r"F:\fsy\project\kae_process\kae_text\sad"
os.makedirs(text_dir, exist_ok=True)

json_paths = [
    r"F:\fsy\project\kae_process\kae_text\sad1.json",
    r"F:\fsy\project\kae_process\kae_text\sad2.json",
    r"F:\fsy\project\kae_process\kae_text\sad3.json",
    r"F:\fsy\project\kae_process\kae_text\sad4.json",
]

# 读取 4 个 json，每个都是 list[str]
all_lists = []
for jp in json_paths:
    with open(jp, "r", encoding="utf-8") as f:
        all_lists.append(json.load(f))

# 简单检查长度一致
lengths = [len(lst) for lst in all_lists]
assert len(set(lengths)) == 1, f"4 个 json 长度不一致: {lengths}"

with open(data, "r", encoding="utf-8") as f:
    names = [ln.strip() for ln in f.readlines()]

assert len(names) == lengths[0], f"all.txt 数量与 json 不一致: {len(names)} vs {lengths[0]}"

for idx, name in enumerate(names):
    # 取出同一 index 的四条描述
    descs = [lst[idx] for lst in all_lists]  # list[str]，长度 4

    # 写一个 txt，4 句各占一行（或你想要的格式）
    out_file = os.path.join(text_dir, name + ".txt")
    with open(out_file, "w", encoding="utf-8") as out_f:
        for d in descs:
            out_f.write(d + "\n")

In [ ]:
#分词 pos

import spacy
nlp = spacy.load("en_core_web_sm")
def pos_tagging(text,start_time=0.0,end_time=0.0):
    """
    纯文本转pos标注格式
    输入："a man kicks something or someone with his right foot"
    输出："a/DET man/NOUN kicks/VERB something/PRON or/CCONJ someone/PRON with/ADP his/PRON right/ADJ foot/NOUN#0.0#0.0"
    """    
    doc = nlp(text.lower())
    tagged = []
    for token in doc:
        pos_map = {
            'ADJ': 'ADJ',
            'ADP': 'ADP',
            'ADV': 'ADV',
            'AUX': 'AUX',
            'CONJ': 'CCONJ',
            'CCONJ': 'CCONJ',
            'DET': 'DET',
            'INTJ': 'INTJ',
            'NOUN': 'NOUN',
            'NUM': 'NUM',
            'PART': 'PART',
            'PRON': 'PRON',
            'PROPN': 'PROPN',
            'PUNCT': 'PUNCT',
            'SCONJ': 'SCONJ',
            'SYM': 'SYM',
            'VERB': 'VERB',
            'X': 'X'
        }
        pos = pos_map.get(token.pos_, 'X')
        tagged.append(f"{token.text}/{pos}")
    tagged_text = ' '.join(tagged) + f"#{start_time}#{end_time}"
    return tagged_text

In [ ]:
text_dir = r"F:\fsy\project\kae_process\kae_text\sad"

for f in os.listdir(text_dir):
    if f.endswith(".txt"):
        file_path = os.path.join(text_dir, f)
        with open(file_path, "r", encoding="utf-8") as in_f:
            lines = [ln.strip() for ln in in_f.readlines() if ln.strip()]

        new_lines = []
        for line in lines:
            tagged_line = pos_tagging(line)

            # 如果原句末尾不是句号，先补一个句号作为分隔
            if not line.endswith("."):
                line_with_sep = line + "."
            else:
                line_with_sep = line

            # 同一行写：原句(+可能补的句号) + POS
            new_lines.append(f"{line_with_sep}{tagged_line}\n")

        with open(file_path, "w", encoding="utf-8") as out_f:
            out_f.writelines(new_lines)